## Packages

Are the requrired packages up and running? If not, install them as described in the setup.

In [ ]:
import pandas
import sklearn
import seaborn
import nltk
import gensim
import transformers
import dotenv
import langchain
import langchain_ollama
import langchain_openai
import langchain_chroma
import langchain_community
import utils
import langgraph
import langchain_huggingface

## APIs

Make sure you saved your API key in the .env file!

### Hugging Face

Check if you have access to the required models. Note that this will download the models.

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from dotenv import load_dotenv
import os

load_dotenv()

This code should give you a sentiment classification:

In [ ]:
distillbert = pipeline(model="distilbert-base-uncased-finetuned-sst-2-english")

distillbert("This workshop will be great!")

This code should give you similar output, but with different labels:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large") 

cap_en = pipeline(task='text-classification', # make classifier pipeline
                      model="poltextlab/xlm-roberta-large-english-media-cap-v3", 
                      tokenizer=tokenizer,
                      token=os.environ.get("HF_TOKEN")) 

print(cap_en("This is a policy-related text in the media."))

cap_de = pipeline(task='text-classification', # make classifier pipeline
                      model="poltextlab/xlm-roberta-large-german-party-cap-v3", 
                      tokenizer=tokenizer,
                      token=os.environ.get("HF_TOKEN")) 

print(cap_de("Das ist ein Parteiprogramm."))

## OpenAI

Check if your OpenAI key works. This code should give you an API response. The `message.content` should read *positive*.

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")) # client = communicates with API

instructions = """ 
You are a sentiment classification system. 

You will be given a sentence and you have to determine if the sentence is "positive", "neutral", or "negative".

Only reply with "positive", "neutral", or "negative".
""" # the triple quotes allow us to have a multi-line string

messages = [
    # we pass the instructions as system message
    {
      "role": "system",
      "content": instructions
    },
    # and the content we wish to classify as user message
    {
      "role": "user",
      "content": "I love this movie!"
    }
]

response = client.chat.completions.create( # completions = create responses
    model="gpt-4o",
    messages=messages
)

print(response)

print(response.choices[0].message.content)

## Ollama

Check if Ollama is installed. If you get a Warning about not connecting to a running Ollama instance, this is okay. Client version should be 0.5.4 (or higher).

In [ ]:
!ollama -v

## CUDA

**Optional**. This checks if your CUDA device is recognized. If not, make sure you installed the correct torch version shipped with CUDA (see setup). If you do not have a CUDA device (usually a NVIDIA GPU) on your machine, you can skip this.

In [ ]:
import torch

if torch.cuda.is_available(): # CUDA is the preferred device
    device = 'cuda:0' # note that if you have multiple GPUs, you might want to change this value to not use the first one
    print(f'Cuda Found. Running on {torch.cuda.get_device_name(0)}')
else: 
    if torch.backends.mps.is_available(): # MPS backend for Mac
        device = 'mps'
        print('MPS Found. Running on MPS')
    else: 
        device = 'cpu' # fallback to CPU computing
        print('No GPU Found. Running on CPU')